In [53]:
# Dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import locale
from locale import atof

In [54]:
# Load in csv
filepath = os.path.join("CSV Files\\TWFC CSV\\Employment.csv")
employment_numbers_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\\TWFC CSV\\Un_numbers.csv")
unemployment_numbers_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\\TWFC CSV\\Un_rate.csv")
unemployment_rate_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\TWFC CSV\\Laborforce.csv")
laborforce_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\\TWFC CSV\\Income.csv")
income_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\\TWFC CSV\\Population.csv")
population_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\\Zillow CSV\\Price.csv")
price_df = pd.read_csv(filepath)
filepath = os.path.join("CSV Files\\Zillow CSV\\Rent.csv")
rent_df = pd.read_csv(filepath)

In [55]:
# Merge the monthly CSVs
merge_df = pd.merge(employment_numbers_df, unemployment_numbers_df, how="outer", on=["Year", "Period ID", "Period", "Area"])
merge_df2 = pd.merge(merge_df, unemployment_rate_df, how="outer", on=["Year", "Period ID", "Period", "Area"])
merge_df3 = pd.merge(merge_df2, laborforce_df, how="outer", on=["Year", "Period ID", "Period", "Area"])

In [56]:
# Select desired columns 
merge_df = merge_df3.drop(merge_df3.columns[[2,4,6,8,10]], axis=1)

# Change column name of Area to City
merge_df=merge_df.rename(columns = {'Area':'City', 'Period ID': 'Month'})

# Slice out MSA out of the City Names
merge_df["City"] = merge_df["City"].replace(' MSA','',regex=True)
merge_df = merge_df

# Convert Strings to Floats
merge_df["Employment"] = merge_df["Employment"].replace(',','',regex=True).astype('float')
merge_df["Unemployment"] = merge_df["Unemployment"].replace(',','',regex=True).astype('float')
merge_df["Labor Force"] = merge_df["Labor Force"].replace(',','',regex=True).astype('float')

#Clean up city names
merge_df['City'], merge_df['x'] = merge_df["City"].str.split('-', 2).str[0:2].str
merge_df = merge_df.drop(merge_df.columns[[-1]], axis=1)

In [57]:
# Convert Region Name into Seperate City and State Columns
rent_df['City'],rent_df['State'] = rent_df["RegionName"].str.split(',', 2).str[0:2].str
rent_df['State'].replace(' TX','TX', inplace=True)
price_df['City'],price_df['State'] = price_df["RegionName"].str.split(',', 2).str[0:2].str
price_df['State'].replace(' TX','TX', inplace=True)

# Extract only the Texas Data
rent_tx_df = rent_df.loc[rent_df["State"] == "TX", :]
rent_tx_df = rent_tx_df.set_index('City')
price_tx_df = price_df.loc[price_df["State"] == "TX", :]
price_tx_df = price_tx_df.set_index('City')

# Remove Undesired Columns
rent_tx_df = rent_tx_df.drop(rent_tx_df.columns[[0,1,2,-1]], axis=1)
price_tx_df = price_tx_df.drop(price_tx_df.columns[[0,1,-1]], axis=1)

# Transpose Data Frame
rent = rent_tx_df.transpose()
price = price_tx_df.transpose()

#Move the Index
rent.reset_index(level=0, inplace=True)
rent = rent.rename(columns = {'index':'Date'})
rent = rent.reset_index(drop=True)
price.reset_index(level=0, inplace=True)
price = price.rename(columns = {'index':'Date'})
price = price.reset_index(drop=True)

# Convert the Date into a Month and Year Column
rent['Year'],rent['Month'] = rent["Date"].str.split('-', 2).str[0:2].str
rent = rent.drop(rent.columns[[0]], axis=1)
price['Year'],price['Month'] = price["Date"].str.split('-', 2).str[0:2].str
price = price.drop(price.columns[[0]], axis=1)

# Melt Data Frames and Restructure and Merge
rent_melt = pd.melt(rent, id_vars=["Year", "Month"])
rent_melt = rent_melt.rename(columns={'value': 'Rent'})
rent_melt = rent_melt.replace('Dallas-Fort Worth', 'Dallas')
price_melt = pd.melt(price, id_vars=["Year", "Month"])
price_melt = price_melt.rename(columns={'value': 'House Price'})
merge_melts = pd.merge(rent_melt,price_melt, how='left', on=['Year','Month','City'])

In [58]:
# Merge TWFC Data with Zillow Data
monthly_merge = pd.merge(merge_df, merge_melts, how="outer", on=["Year", "Month", "City"])

In [59]:
# Clean up Income data to merge to annual data frame
# Select desired columns 
income_df = income_df.drop(income_df.columns[[1,3,4]], axis=1)

# Change column name of Area to City
income_df = income_df.rename(columns = {'Area':'City'})

# Slice out MSA out of the City Names
income_df["City"] = income_df["City"].replace(' MSA','',regex=True)

#Clean up city names
income_df['City'], income_df['x'] = income_df["City"].str.split('-', 2).str[0:2].str
income_df = income_df.drop(income_df.columns[[-1]], axis=1)

# Convert Strings to Floats
income_df["Income"] = income_df["Income"].replace(',','',regex=True)

income = []
for x in income_df["Income"]:
    x = x[1:]
    x = int(x)
    income.append(x)

income_df["Income"] = income

# Merge Income Data Frame to the Main Data Frame
df_final = pd.merge(monthly_merge, income_df, how="outer", on=["City", "Year"])
df_final['City'].unique()

array(['Abilene', 'Amarillo', 'Austin', 'Beaumont', 'Brownsville',
       'College Station', 'Corpus Christi', 'Dallas', 'El Paso', 'Houston',
       'Killeen', 'Laredo', 'Longview', 'Lubbock', 'McAllen', 'Midland',
       'Odessa', 'San Angelo', 'San Antonio', 'Sherman', 'Texarkana',
       'Tyler', 'Victoria', 'Waco', 'Wichita Falls', 'Huntsville',
       'Marshall', 'Palestine', 'Eagle Pass', 'Jacksonville', 'Kerrville',
       'Del Rio', 'Corsicana', 'El Campo', 'Alice', 'Gainesville',
       'Brownwood', 'Stephenville', 'Plainview', 'Big Spring',
       'Sulphur Springs', 'Brenham', 'Kingsville', 'Fredericksburg',
       'Pampa', 'Dumas', 'Snyder', 'Sweetwater', 'Andrews', 'Pecos',
       'Vernon'], dtype=object)

In [60]:
df_final = df_final.groupby(['City', 'Year']).mean()
df_final = df_final.drop(df_final.columns[[0]], axis=1)